# Solution: MF of an explicit feedback (ratings) matrix

**Given:**
- data loader and matrix initialization code (step1+2)
- default params settings, training and evaluation (steps 5 to 8)

**Answer the following questions:**
- Q1: Implement L2 and L2 losses and evaluate with default params (step 3)
- Q2: Implement L1 and L2 regularizations with default params (step 4)
- Q3: Bench the parameters around the default values

**Original source:** http://hameddaily.blogspot.fr/2016/12/simple-matrix-factorization-with.html

### Step 1: Load the data

In [17]:
import numpy
import tensorflow as tf
import pandas as pd

print(tf.__version__)
# read data
df = pd.read_csv('data/ml-100k/u.data', sep='\t', names=['user', 'item', 'rate', 'time'])
numpy.random.seed(42)
msk = numpy.random.rand(len(df)) < 0.7
df_train = df[msk]
df_test = df[~msk]

user_index = [x-1 for x in df_train.user.values]
item_index = [x-1 for x in df_train.item.values]
user_index_test = [x-1 for x in df_test.user.values]
item_index_test = [x-1 for x in df_test.item.values]

rates = df_train.rate.values
rates_test = df_test.rate.values
num_ratings = len(rates)
num_ratings_test = len(rates_test)
mean_rating = numpy.mean (rates)

print ("Mean (train) rating = " + str(mean_rating))
print ("Number of ratings (train/val/total) = " + str(num_ratings) + "/" + str(num_ratings_test) + "/" + str(num_ratings + num_ratings_test))

1.8.0
Mean (train) rating = 3.529004007930282
Number of ratings (train/val/total) = 70111/29889/100000


### Step 2: Define/initialize the User and Item matrices and use their product to compute initial ratings R

In [18]:
# variables
feature_len = 10
num_users = 943
num_items = 1682

U = tf.Variable(initial_value=tf.truncated_normal([num_users,feature_len]), name='users')
P = tf.Variable(initial_value=tf.truncated_normal([feature_len,num_items]), name='items')
result = tf.matmul(U, P)
result_flatten = tf.reshape(result, [-1])

# rating
result_values = tf.gather(result_flatten, user_index * tf.shape(result)[1] + item_index, name='predicted_ratings')
result_values_test = tf.gather(result_flatten, user_index_test * tf.shape(result)[1] + item_index_test, name='validation_ratings')


### Step 3: Define the cost function (try L2, L1)

In [19]:
# Calculate the difference between the predicted ratings and the actual
# ratings. The predicted ratings are the values obtained form the matrix
# multiplication with the mean rating added on.

# L1 cost function
#diff_op = tf.subtract(tf.add(result_values, mean_rating, name="add_mean"), rates, name='trainig_diff')
#diff_op_abs = tf.square(diff_op, name="abs_difference")
#base_cost = tf.reduce_sum(diff_op_abs, name="sum_abs_error")

# L2 cost function
diff_op = tf.subtract(tf.add(..., ..., name="add_mean"), rates, name='trainig_diff')
diff_op_squared = tf.square(..., name="squared_difference")
base_cost = tf.reduce_sum(..., name="sum_squared_error")



### Step 4: Add regularization term for User, Item matrices (try L2, L1)

In [25]:
# L1 regularization
l1_norm_sums = tf.add(tf.reduce_sum(tf.abs(..., name='user_abs'), name='user_norm'),
                   tf.reduce_sum(tf.abs(..., name='item_abs'), name='item_norm'))

# L2 regularization
l2_norm_sums = tf.add(tf.reduce_sum(tf.square(..., name='user_abs'), name='user_norm'),
                   tf.reduce_sum(tf.square(..., name='item_abs'), name='item_norm'))

# regularized cost
with tf.name_scope("training_cost") as scope:
    lamda = tf.constant(.001, name='lambda')
    regularizer = tf.multiply(..., ..., 'regularizer')
    regularized_cost = tf.add(..., ...)
    tf.summary.scalar('reg_cost', regularized_cost)
    
# test cost
diff_op_test = tf.subtract(tf.add(result_values_test, mean_rating, name="add_mean"), rates_test, name='trainig_diff_test')
with tf.name_scope("validation_cost") as scope:
    cost_test = tf.div(tf.reduce_sum(tf.square(diff_op_test, name="squared_difference_test"), name="sum_squared_error_test"), num_ratings_test * 2, name="average_error")


### Step 5: Setup training (learning rate, optimizer)

In [21]:
# Define cost function

with tf.name_scope("train") as scope:
    lr = tf.constant(.001, name='learning_rate')
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(lr, global_step, 10000, 0.96, staircase=True)
    optimizer = tf.train. ...
    training_step = optimizer.minimize(..., global_step=global_step)

### Step 6: Accuracy

In [22]:
# Define accuracy
threshold = 1.0
with tf.name_scope("training_accuracy") as scope:
  # Just measure the absolute difference against the threshold
  good = tf.less(tf.abs(diff_op), threshold)

  accuracy_tr = tf.div(tf.reduce_sum(tf.cast(good, tf.float32)), num_ratings)
  
with tf.name_scope("validation_accuracy") as scope:
  # Validation set accuracy:
  good_test = tf.less(tf.abs(diff_op_test), threshold)
  accuracy_test = tf.reduce_sum(tf.cast(good_test, tf.float32)) / num_ratings_test
  

### Step 7: Run Training

In [23]:
#Run training
# execute
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

num_epochs = 300
for i in range(num_epochs):
    if (i%100==0):
        res = sess.run([..., ..., ..., ...])
        acc_tr = res[0]
        acc_test = res[1]
        cost_ev = res[2]
        cost_test_ev = res[3]
        print("Training accuracy at step %s: %s" % (i, acc_tr))
        print("Validation accuracy at step %s: %s" % (i, acc_test))
        print("Training cost: %s" % (cost_ev))
        print("Validation cost: %s" % (cost_test_ev))
    else:
        sess.run ([training_step])

Training accuracy at step 0: 0.2995821
Validation accuracy at step 0: 0.2987721
Training cost: 499905.2
Validation cost: 3.5770955
Training accuracy at step 100: 0.76838154
Validation accuracy at step 100: 0.6555589
Training cost: 51360.707
Validation cost: 0.656112
Training accuracy at step 200: 0.82099813
Validation accuracy at step 200: 0.6731908
Training cost: 40853.754
Validation cost: 0.6345121


### Step 8: Run Evaluation

In [24]:
# example
u, p, r = df[['user', 'item', 'rate']].values[1]
rhat = tf.gather(tf.gather(tf.add(result, mean_rating), u-1), p-1)
print ("rating for user " + str(u) + " for item " + str(p) + " is " + str(r) + " and our prediction is: " + str(sess.run(rhat)))

# RMSE
rmse_cost_test = tf.sqrt(tf.div(tf.reduce_sum(tf.square(diff_op_test, name="squared_difference_test"), name="sum_squared_error_test"), df_test.shape[0] * 2, name="average_error"))

print ("RMSE:" + str(sess.run(rmse_cost_test)))

rating for user 186 for item 302 is 3 and our prediction is: 4.1490483
RMSE:0.79964453
